In [1]:
#pip install transformers[torch]

In [2]:
#pip install datasets

In [3]:
from datasets import load_dataset

df = load_dataset("doushabao4766/msra_ner_k_V3")

d:\software\python\miniconda\envs\py312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# import pandas as pd

# splits = {'train': 'data/train-00000-of-00001-42717a92413393f9.parquet', 'test': 'data/test-00000-of-00001-8899cab5fdab45bc.parquet'}
# df = pd.read_parquet("hf://datasets/doushabao4766/msra_ner_k_V3/" + splits["train"])
# df_test = pd.read_parquet("hf://datasets/doushabao4766/msra_ner_k_V3/" + splits["test"])

In [5]:
# df.head
# df.columns
# df

In [6]:
# 验证tag标签数量
# tags_id = set()
# for item in df['ner_tags']:
#     tags_id.update(item)  
# tags_id




In [7]:
# 这是自定义的，分别为人物地点与组织机构
entites = ['O'] + list({'PER', 'LOC', 'ORG'})
tags = ['O']
for entity in entites[1:]:
    tags.append('B-' + entity.upper())
    tags.append('I-' + entity.upper())

entity_index = {entity:i for i, entity in enumerate(entites)}

In [8]:
#开始分词

# item['tokens']	输入的是已经按字符分割好的列表，如 ['北', '京', ...]
# is_split_into_words=True	表示输入已经是分词后的列表（不是完整句子）
# add_special_tokens=False	不自动添加 [CLS], [SEP] 等特殊标记（你自己会处理）
# truncation=True 和 max_length=512	超过 512 的 token 会被截断
# return_offsets_mapping=True	返回每个 token 在原文中的位置映射（调试用，非必须）

from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification

tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')


def data_input_proc(item):
    input_data = tokenizer(item['tokens'], 
                           truncation=True,
                           add_special_tokens=False, 
                           max_length=512, 
                           is_split_into_words=True,
                           return_offsets_mapping=True)
    #限制长度只能是512以内
    labels = [lbl[:512] for lbl in item['ner_tags']]
    input_data['labels'] = labels
    return input_data

#batched=True	可选，表示每次批量处理多个样本，提升效率（但在这个函数里可能不需要）
ds1 = df.map(data_input_proc, batched=True)


In [9]:
ds_train=ds1['train']
ds_test=ds1['test']

ds_train['labels']

Column([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 3, 4, 4, 0, 5, 6, 6, 6, 6, 6, 6, 0, 5, 6, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [10]:
# 记录转换为pytorch
#input_ids为词id，token_type_ids为句子id，attention_mask用于标注出哪些是填充的
ds1.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
#第二步，设置超参数
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
    output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
    num_train_epochs = 3,    # 训练 epoch
    save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
    per_device_train_batch_size=32,  # 训练批次
    per_device_eval_batch_size=32,
    report_to='tensorboard',  # 训练输出记录，默认为WANDB，这里指定为了tensorboard
    eval_strategy="epoch",
)


 #构建一个用于命名实体识别（NER）任务的 BERT 模型，并配置标签 ID 与标签名称之间的映射
id2lbl = {i:tag for i, tag in enumerate(tags)}
lbl2id = {tag:i for i, tag in enumerate(tags)}

model = AutoModelForTokenClassification.from_pretrained('bert-base-chinese', 
                                                        num_labels=len(tags),
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)



Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import evaluate  
import numpy as np
#在命名实体识别（NER）任务中，使用 seqeval 指标来评估模型预测结果的质量。
# metric 方法
def compute_metric(result):
    # result 是一个tuple (predicts, labels)
    
    # 获取评估对象
    seqeval = evaluate.load('seqeval')
    predicts,labels = result
    predicts = np.argmax(predicts, axis=2)
    
    # 准备评估数据
    predicts = [[tags[p] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    results = seqeval.compute(predictions=predicts, references=labels)

    return results

In [13]:
#处理输入与标签，保证他们数据一致
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)


In [15]:
#第三步，创建模型
trainer = Trainer(
    model,
    args,
    train_dataset=ds1['train'],
    eval_dataset=ds1['test'],
    data_collator=data_collator,
    compute_metrics=compute_metric
)
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
#推理部分
from transformers import pipeline
pipeline = pipeline('token-classification', 'msra_ner_train/checkpoint-2112')
pipeline('双方确定了今后发展中美关系的指导方针')